In [1]:
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots

from typing import List

import model_fe

degree_sign = u'\N{DEGREE SIGN}'

2023-07-10 12:48:53.617631: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
ft_percents = [0.0, 0.2, 0.4, 0.6, 0.8]
shuffle_seeds = [17, 42, 64, 76, 98]

In [3]:
def plot_multiseed_error_pir(ft_percents: List[float], from_building: str, from_tower: int, to_building: str, season: str = None):
    # MAKE PREDICTIONS AND PLOT TRENDS
    rmse_list = []
    # [tow-1] [seed index] [error]

    for tow in range(1,3):
        to_features = [f'{to_building}_Tower_{tow} enteringWaterTemp', f'{to_building}_Tower_{tow} outdoorAirDryBulb', f'{to_building}_Tower_{tow} outdoorAirWetBulb', f'{to_building}_Tower_{tow} vfdPercent', f'{to_building}_Tower_{tow} fanA_vfdPower', f'{to_building}_Tower_{tow} fanB_vfdPower', f'{to_building}_Tower_{tow} dayOfWeek', f'{to_building}_Tower_{tow} hourOfDay', f'{to_building}_Tower_{tow} efficiency']
        to_target = f'{to_building}_Tower_{tow} leavingWaterTemp'
        rmse_list.append([])

        for s, seed in enumerate(shuffle_seeds):
            rmse_list[tow-1].append([])

            for ft_percent in ft_percents:
                rmse, fig, mabs = model_fe.feature_extr_transfer(from_building_name=from_building, from_tower_number=from_tower, to_building_name=to_building, to_tower_number=tow, to_features=to_features, to_target=to_target, to_season=season, finetuning_percentage=ft_percent, display_results=False, use_delta=True, shuffle_seed=seed)
                rmse_list[tow-1][s].append(rmse)


    # PLOT ERROR GRADIENT

    # tower 1: create a DataFrame and line graph using Plotly Express
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 
                                f'RMSE with seed {shuffle_seeds[0]}': rmse_list[0][0], 
                                f'RMSE with seed {shuffle_seeds[1]}': rmse_list[0][1],
                                f'RMSE with seed {shuffle_seeds[2]}': rmse_list[0][2],
                                f'RMSE with seed {shuffle_seeds[3]}': rmse_list[0][3],
                                f'RMSE with seed {shuffle_seeds[4]}': rmse_list[0][4],})
    fig1 = px.line(print_df, y=[f'RMSE with seed {seed}' for seed in shuffle_seeds], x="finetuning data percentage", title='Errors')
    # tower 2: create a DataFrame and line graph using Plotly Expres
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 
                                f'RMSE with seed {shuffle_seeds[0]}': rmse_list[0][0], 
                                f'RMSE with seed {shuffle_seeds[1]}': rmse_list[0][1],
                                f'RMSE with seed {shuffle_seeds[2]}': rmse_list[0][2],
                                f'RMSE with seed {shuffle_seeds[3]}': rmse_list[0][3],
                                f'RMSE with seed {shuffle_seeds[4]}': rmse_list[0][4],})
    fig2 = px.line(print_df, y=[f'RMSE with seed {seed}' for seed in shuffle_seeds], x="finetuning data percentage", title='Errors')

    figfinal = make_subplots(rows=1, cols=2)
    for dt in fig1.data:
        figfinal.add_trace(dt, row=1, col=1)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=1)
    figfinal.update_yaxes(title_text="Error", row=1, col=1)

    for dt in fig2.data:
        figfinal.add_trace(dt, row=1, col=2)
    figfinal.add_trace(fig2.data[1], row=1, col=2)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=2)
    figfinal.update_yaxes(title_text="Error", row=1, col=2)

    figfinal.update_layout(title_text=f"{from_building} Tower {from_tower} to {to_building} Tower 1, {from_building} Tower {from_tower} to {to_building} Tower 2 ({season}) (feature extraction)", showlegend=True)

    figfinal.show()
    figfinal.write_html(f"../plots/feature_extraction/interbuilding_transfers/{from_building}{from_tower}_to_{to_building}_gradient_{season}.html")

    # PLOT PERFORMANCE IMPROVEMENT RATIO

    pir = [[[(seed_rmses[0] - rmse) / seed_rmses[0] for rmse in seed_rmses] for seed_rmses in tower_rmses] for tower_rmses in rmse_list]

    # tower 1: create a DataFrame and line graph using Plotly Express
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 
                                f'PIR with seed {shuffle_seeds[0]}': pir[0][0], 
                                f'PIR with seed {shuffle_seeds[1]}': pir[0][1],
                                f'PIR with seed {shuffle_seeds[2]}': pir[0][2],
                                f'PIR with seed {shuffle_seeds[3]}': pir[0][3],
                                f'PIR with seed {shuffle_seeds[4]}': pir[0][4],})
    fig1 = px.line(print_df, y=[f'PIR with seed {seed}' for seed in shuffle_seeds], x="finetuning data percentage", title='Performance improvement ratio')

    # tower 2: create a DataFrame and line graph using Plotly Expres
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 
                                f'PIR with seed {shuffle_seeds[0]}': pir[1][0], 
                                f'PIR with seed {shuffle_seeds[1]}': pir[1][1],
                                f'PIR with seed {shuffle_seeds[2]}': pir[1][2],
                                f'PIR with seed {shuffle_seeds[3]}': pir[1][3],
                                f'PIR with seed {shuffle_seeds[4]}': pir[1][4],})
    fig2 = px.line(print_df, y=[f'PIR with seed {seed}' for seed in shuffle_seeds], x="finetuning data percentage", title='Performance improvement ratio')

    figfinal = make_subplots(rows=1, cols=2)
    for dt in fig1.data:
        figfinal.add_trace(dt, row=1, col=1)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=1)
    figfinal.update_yaxes(title_text="Performance improvement ratio", row=1, col=1)

    for dt in fig2.data:
        figfinal.add_trace(dt, row=1, col=2)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=2)
    figfinal.update_yaxes(title_text="Performance improvement ratio", row=1, col=2)

    figfinal.update_layout(title_text=f"{from_building} Tower {from_tower} to {to_building} Tower 1, {from_building} Tower {from_tower} to {to_building} Tower 2 ({season}) (feature extraction)", showlegend=True)

    figfinal.show()
    figfinal.write_html(f"../plots/feature_extraction/interbuilding_transfers/{from_building}{from_tower}_to_{to_building}_pir_{season}.html")

In [4]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="ESB", from_tower=1, to_building="Kissam", season="summer")

Tower 1 first temp: 79.07764434814453


2023-07-10 12:48:58.212178: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


803/803 [==============================] - 1s 991us/step
Tower 1 first temp: 79.07764434814453
finetuning_percentage: 0.2 vec_X_train.shape: (5138, 6, 10), vec_X_test.shape: (20555, 6, 10), vec_y_train.shape: (5138,), vec_y_test.shape: (20555,)
643/643 [==============================] - 1s 939us/step
Tower 1 first temp: 79.07764434814453
finetuning_percentage: 0.4 vec_X_train.shape: (10277, 6, 10), vec_X_test.shape: (15416, 6, 10), vec_y_train.shape: (10277,), vec_y_test.shape: (15416,)
482/482 [==============================] - 1s 1ms/step
Tower 1 first temp: 79.07764434814453
finetuning_percentage: 0.6 vec_X_train.shape: (15415, 6, 10), vec_X_test.shape: (10278, 6, 10), vec_y_train.shape: (15415,), vec_y_test.shape: (10278,)
322/322 [==============================] - 1s 957us/step
Tower 1 first temp: 79.07764434814453
finetuning_percentage: 0.8 vec_X_train.shape: (20554, 6, 10), vec_X_test.shape: (5139, 6, 10), vec_y_train.shape: (20554,), vec_y_test.shape: (5139,)
161/161 [=========

In [5]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="ESB", from_tower=2, to_building="Kissam", season="summer")

Tower 1 first temp: 79.07764434814453
803/803 [==============================] - 1s 947us/step
Tower 1 first temp: 79.07764434814453
finetuning_percentage: 0.2 vec_X_train.shape: (5138, 6, 10), vec_X_test.shape: (20555, 6, 10), vec_y_train.shape: (5138,), vec_y_test.shape: (20555,)
643/643 [==============================] - 1s 971us/step
Tower 1 first temp: 79.07764434814453
finetuning_percentage: 0.4 vec_X_train.shape: (10277, 6, 10), vec_X_test.shape: (15416, 6, 10), vec_y_train.shape: (10277,), vec_y_test.shape: (15416,)
482/482 [==============================] - 1s 978us/step
Tower 1 first temp: 79.07764434814453
finetuning_percentage: 0.6 vec_X_train.shape: (15415, 6, 10), vec_X_test.shape: (10278, 6, 10), vec_y_train.shape: (15415,), vec_y_test.shape: (10278,)
322/322 [==============================] - 1s 991us/step
Tower 1 first temp: 79.07764434814453
finetuning_percentage: 0.8 vec_X_train.shape: (20554, 6, 10), vec_X_test.shape: (5139, 6, 10), vec_y_train.shape: (20554,), vec_

In [6]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="Kissam", from_tower=1, to_building="ESB", season="summer")

Tower 1 first temp: 79.19134521484375
449/449 [==============================] - 1s 1ms/step
Tower 1 first temp: 79.19134521484375
finetuning_percentage: 0.2 vec_X_train.shape: (2868, 6, 10), vec_X_test.shape: (11474, 6, 10), vec_y_train.shape: (2868,), vec_y_test.shape: (11474,)
359/359 [==============================] - 1s 1ms/step
Tower 1 first temp: 79.19134521484375
finetuning_percentage: 0.4 vec_X_train.shape: (5736, 6, 10), vec_X_test.shape: (8606, 6, 10), vec_y_train.shape: (5736,), vec_y_test.shape: (8606,)
269/269 [==============================] - 1s 1ms/step
Tower 1 first temp: 79.19134521484375
finetuning_percentage: 0.6 vec_X_train.shape: (8605, 6, 10), vec_X_test.shape: (5737, 6, 10), vec_y_train.shape: (8605,), vec_y_test.shape: (5737,)
180/180 [==============================] - 0s 1ms/step
Tower 1 first temp: 79.19134521484375
finetuning_percentage: 0.8 vec_X_train.shape: (11473, 6, 10), vec_X_test.shape: (2869, 6, 10), vec_y_train.shape: (11473,), vec_y_test.shape: (2

In [7]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="Kissam", from_tower=2, to_building="ESB", season="summer")

Tower 1 first temp: 79.19134521484375
449/449 [==============================] - 1s 1ms/step
Tower 1 first temp: 79.19134521484375
finetuning_percentage: 0.2 vec_X_train.shape: (2868, 6, 10), vec_X_test.shape: (11474, 6, 10), vec_y_train.shape: (2868,), vec_y_test.shape: (11474,)
359/359 [==============================] - 1s 1ms/step
Tower 1 first temp: 79.19134521484375
finetuning_percentage: 0.4 vec_X_train.shape: (5736, 6, 10), vec_X_test.shape: (8606, 6, 10), vec_y_train.shape: (5736,), vec_y_test.shape: (8606,)
269/269 [==============================] - 1s 1ms/step
Tower 1 first temp: 79.19134521484375
finetuning_percentage: 0.6 vec_X_train.shape: (8605, 6, 10), vec_X_test.shape: (5737, 6, 10), vec_y_train.shape: (8605,), vec_y_test.shape: (5737,)
180/180 [==============================] - 0s 1ms/step
Tower 1 first temp: 79.19134521484375
finetuning_percentage: 0.8 vec_X_train.shape: (11473, 6, 10), vec_X_test.shape: (2869, 6, 10), vec_y_train.shape: (11473,), vec_y_test.shape: (2

## --- Testing on MRB ---

In [8]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="ESB", from_tower=1, to_building="MRB", season="summer")

Tower 1 first temp: 81.48371124267578
501/501 [==============================] - 1s 932us/step
Tower 1 first temp: 81.48371124267578
finetuning_percentage: 0.2 vec_X_train.shape: (3206, 6, 10), vec_X_test.shape: (12824, 6, 10), vec_y_train.shape: (3206,), vec_y_test.shape: (12824,)
401/401 [==============================] - 1s 977us/step
Tower 1 first temp: 81.48371124267578
finetuning_percentage: 0.4 vec_X_train.shape: (6412, 6, 10), vec_X_test.shape: (9618, 6, 10), vec_y_train.shape: (6412,), vec_y_test.shape: (9618,)
301/301 [==============================] - 1s 973us/step
Tower 1 first temp: 81.48371124267578
finetuning_percentage: 0.6 vec_X_train.shape: (9618, 6, 10), vec_X_test.shape: (6412, 6, 10), vec_y_train.shape: (9618,), vec_y_test.shape: (6412,)
201/201 [==============================] - 0s 1ms/step
Tower 1 first temp: 81.48371124267578
finetuning_percentage: 0.8 vec_X_train.shape: (12824, 6, 10), vec_X_test.shape: (3206, 6, 10), vec_y_train.shape: (12824,), vec_y_test.sha

In [9]:
plot_multiseed_error_pir(ft_percents=ft_percents, from_building="Kissam", from_tower=1, to_building="MRB", season="summer")

Tower 1 first temp: 81.48371124267578
501/501 [==============================] - 1s 960us/step
Tower 1 first temp: 81.48371124267578
finetuning_percentage: 0.2 vec_X_train.shape: (3206, 6, 10), vec_X_test.shape: (12824, 6, 10), vec_y_train.shape: (3206,), vec_y_test.shape: (12824,)
401/401 [==============================] - 1s 963us/step
Tower 1 first temp: 81.48371124267578
finetuning_percentage: 0.4 vec_X_train.shape: (6412, 6, 10), vec_X_test.shape: (9618, 6, 10), vec_y_train.shape: (6412,), vec_y_test.shape: (9618,)
301/301 [==============================] - 1s 977us/step
Tower 1 first temp: 81.48371124267578
finetuning_percentage: 0.6 vec_X_train.shape: (9618, 6, 10), vec_X_test.shape: (6412, 6, 10), vec_y_train.shape: (9618,), vec_y_test.shape: (6412,)
201/201 [==============================] - 0s 995us/step
Tower 1 first temp: 81.48371124267578
finetuning_percentage: 0.8 vec_X_train.shape: (12824, 6, 10), vec_X_test.shape: (3206, 6, 10), vec_y_train.shape: (12824,), vec_y_test.s